In [2]:
from six.moves import cPickle as pickle
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [22]:
data = pd.read_csv('data/in-2017-10-11.csv')

In [23]:
pickle_file = 'data/preprocessing.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    X1_lookup = save['X1_lookup']
    X2_mean = save['X2_mean']
    X2_std = save['X2_std']
    print('Model Loaded')

Model Loaded


In [25]:
X1_raw = data['master_id']
X2_raw = data.drop(['master_id'], axis=1)

X2 = (X2_raw - X2_mean)/X2_std
X2 = X2.fillna(0)

In [26]:
X1 = pd.DataFrame(columns=['master_idx'], index = X1_raw.index)

for i in tqdm(range(len(X1_raw))):
    if X1_raw[i] in X1_lookup:
        X1['master_idx'][i] = X1_lookup.index(X1_raw[i])
    else:
        X1['master_idx'][i] = 0

100%|██████████| 77813/77813 [00:42<00:00, 1815.41it/s]


In [27]:
from keras.layers import Input, Embedding, merge
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers.merge import concatenate
from sklearn.metrics import roc_auc_score
n_masters = len(X1_lookup)
n_factors = 50

master_in = Input(shape=(1,), dtype='int64', name='master_in')
m1 = Embedding(n_masters, n_factors, input_length=1)(master_in)
m2 = Flatten()(m1)
order_in = Input(shape=(12,), dtype='float32', name='order_in')

x = concatenate([m2, order_in])
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(100, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model([master_in, order_in], x)
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])

In [28]:
model.load_weights('data/model_01.h5')

In [29]:
X1 = X1.as_matrix()
X2 = X2.as_matrix()

y_pred = model.predict([X1, X2])

ValueError: Error when checking : expected order_in to have shape (None, 12) but got array with shape (77813, 13)

In [21]:
y_pred

array([[ 0.51477754],
       [ 0.52960706],
       [ 0.50997615],
       [ 0.48450628],
       [ 0.4031713 ],
       [ 0.77555501],
       [ 0.42649108],
       [ 0.35156983],
       [ 0.39891058],
       [ 0.8020969 ],
       [ 0.79594707],
       [ 0.66114855],
       [ 0.52950144],
       [ 0.76292104],
       [ 0.5039283 ],
       [ 0.53319186],
       [ 0.62876862],
       [ 0.75524354],
       [ 0.46127051],
       [ 0.24147557],
       [ 0.64058518],
       [ 0.45217055],
       [ 0.55702049],
       [ 0.32888082],
       [ 0.75662738],
       [ 0.17107491],
       [ 0.26774871],
       [ 0.44999844],
       [ 0.66588491],
       [ 0.68139756],
       [ 0.61514884],
       [ 0.49812728],
       [ 0.77845073],
       [ 0.51126784],
       [ 0.3674745 ],
       [ 0.39409909],
       [ 0.39958954],
       [ 0.69923788],
       [ 0.49586311],
       [ 0.83687037],
       [ 0.40976399],
       [ 0.69510978],
       [ 0.39164793],
       [ 0.78712583],
       [ 0.27155   ],
       [ 0